In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-preparation/purches_cols203_v3.csv
/kaggle/input/data-preparation/__results__.html
/kaggle/input/data-preparation/__notebook__.ipynb
/kaggle/input/data-preparation/__output__.json
/kaggle/input/data-preparation/purches.csv
/kaggle/input/data-preparation/custom.css
/kaggle/input/uplift-shift-23/baseline.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/train_purch/train_purch.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/products.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/clients2.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/train.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/test.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/test_purch/test_purch.csv


In [2]:
%%time
test_purch = pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/test_purch/test_purch.csv').set_index('client_id', drop=False)
train_purch = pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/train_purch/train_purch.csv').set_index('client_id', drop=False)
clients = pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/data/clients2.csv').set_index('client_id', drop=False)
train = pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/data/train.csv').set_index('client_id', drop=False)
products = pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/data/products.csv')

CPU times: user 39.6 s, sys: 7.57 s, total: 47.2 s
Wall time: 1min 11s


#### 1) Каково значение аплифта на всем трейне?

In [3]:
def uplift(df, purch_col = 'purchased', treat_col = 'treatment_flg'):
    return (df.loc[df['treatment_flg']==1, 'purchased'].mean() - df.loc[df['treatment_flg']==0, 'purchased'].mean()) * 100

In [4]:
# ровного такого варианта там нет (это странно, т.к. в других вопросах эта же функция дает посчитать точный ответ)
uplift(train)

3.354516795221374

#### 2) Выберите правильные утверждения:

In [5]:
# ВОПРОС: В трейне мужчин больше чем женщин. 
# ОТВЕТ: False
for name, group in clients.loc[train.index].groupby('gender'):
    print(name, group.shape[0])

F 51559
M 23503
U 64965


In [6]:
# ВОПРОС: Медианный возраст всех людей в трейне равен 39 годам.
# ОТВЕТ: False
clients.loc[train.index]['age'].median()

45.0

In [7]:
# ВОПРОС: Аплифт в группе людей, возраст которых больше 50 лет, 
# примерно в два раза выше чем у остальных людей
# ОТВЕТ: True
train_clients = clients.loc[train.index]
more50 = train_clients[train_clients['age'] > 50].index
less50 = train_clients[train_clients['age'] <= 50].index

print(uplift(train.loc[more50]), uplift(train.loc[less50]))
print(uplift(train.loc[more50]) / uplift(train.loc[less50]))

5.06140212921522 2.1301489366581716
2.3760789877705304


In [8]:
# ВОПРОС: Последняя покупка в train_purch была совершена в марте 2018 года.
# ОТВЕТ: False

train_purch['transaction_day_str'] = train_purch['transaction_datetime'].progress_apply(lambda x: x[:10])
train_purch['transaction_day_str'].max()

100%|██████████| 15998952/15998952 [00:19<00:00, 830802.76it/s]


'2019-03-18'

#### 3) На какую максимальную сумму в train_purch была совершена покупка? (округлить до целого)

In [9]:
train_purch['purchase_sum'].max()

29611.48

#### 4) Каково мат. ожидание кол-ва посетителей во всех магазин в train_purch в день? (округлить до целого)

In [10]:
# здесь 1 транзакция = 1 посетитель
train_purch.groupby('transaction_day_str')['transaction_id'].nunique().mean()

23855.915254237287

#### 5) Какого значения аплифта можно достичь, если осуществить воздействие на людей, с момента последней покупки которых прошло 10 и более дней?

In [11]:
train_purch['transaction_secs'] = pd.to_datetime(train_purch['transaction_datetime'], format='%Y-%m-%d %H:%M:%S').progress_apply(lambda x: x.timestamp())

100%|██████████| 15998952/15998952 [01:35<00:00, 166699.27it/s]


In [12]:
%%time
last_purches = train_purch.drop('client_id', axis=1).groupby('client_id')['transaction_secs'].max()

CPU times: user 5 s, sys: 2.71 s, total: 7.72 s
Wall time: 7.72 s


In [13]:
cur_time = max(last_purches) # видимо, имеется в виду это
more10days = cur_time - last_purches >= 10 * 24 * 60 * 60

In [14]:
uplift(train.loc[more10days[more10days].index])

3.7241569749273062

#### 6) Какова доля продукции собственного производства в покупках среди всех покупок?

In [15]:
train_purch.drop(columns=['transaction_secs', 'transaction_day_str'], inplace=True)

In [16]:
%%time
purches = test_purch.append(train_purch, ignore_index=True)

CPU times: user 2.84 s, sys: 1.72 s, total: 4.57 s
Wall time: 4.58 s


In [17]:
tmp = pd.merge(products[['product_id', 'is_own_trademark']], purches['product_id'], how='right', on='product_id')
tmp['is_own_trademark'].sum() / tmp.shape[0] * 100

15.239869088817793

#### 7) Влияет ли факт покупки самого популярного продукта на аплифт? Если влияет, то каково значение аплифта?

In [18]:
prod_count = train_purch.groupby('product_id')['transaction_id'].count()
most_pop_prod = prod_count.idxmax()
prod_count.idxmax(), prod_count.max()

('4009f09b04', 636488)

In [19]:
clients_bought_pop_prod = train_purch[train_purch['product_id'] == most_pop_prod].index.unique()

In [20]:
uplift(train.loc[clients_bought_pop_prod])

2.643710546147404

#### 8) Разделим покупателей на две группы: владеют картой покупателя > 100 дней (группа А), владеют картой покупателя <= 100 дней (группа Б). Правда ли то, что аплифт в группе А меньше, чем в группе Б? Если правда, то на сколько меньше (разница между процентами)?

In [21]:
# избавляемся тут с помощью dropna() от тех, у кого карты покупателя нет вообще
clients['get_card_secs'] = pd.to_datetime(clients['first_redeem_date'].dropna(), format='%Y-%m-%d %H:%M:%S').progress_apply(lambda x: x.timestamp())

100%|██████████| 182493/182493 [00:01<00:00, 177563.33it/s]


In [22]:
train_clients = clients.loc[train.index]

In [23]:
more100days = cur_time - train_clients['get_card_secs'].dropna() > 100 * 24 * 60 * 60

In [24]:
A = more100days
B = ~more100days

In [25]:
# неточное совпадение ответа скорее всего потому что я смотрю на точные 10 суток в секундах, а не просто на дату
uplift(train.loc[B[B].index]) - uplift(train.loc[A[A].index])

2.240123502012259

#### 9) Разделим покупателей на две группы: покупали продукты собственного производства хоть раз (группа А), вообще не покупали продукты собственного производства (группа Б). Правда ли то, что аплифт в группе А меньше чем в группе Б? Если правда, то на сколько меньше? (округлить до среднего)

In [26]:
%%time
own_prods = products[products['is_own_trademark'] == 1]['product_id']
clients_bought_own_prods = pd.merge(train_purch[['client_id', 'product_id']], own_prods, how='right', on='product_id')['client_id'].drop_duplicates()

CPU times: user 3.79 s, sys: 216 ms, total: 4.01 s
Wall time: 4.01 s


In [27]:
A_index = train.index.isin(clients_bought_own_prods)
B_index = ~A_index

In [28]:
uplift(train[B_index]) - uplift(train[A_index])

4.728034139476883

#### 10) Разделим покупателей на две группы. В группе А у большей части покупок клиента чек больше среднего чека по всей выборке. В группе Б у  большей части покупок клиента чек меньше среднего чека по всей выборке. Правда ли то, что аплифт в группе А меньше, чем в группе Б?

In [29]:
%%time
mean_bill_sum_train = train_purch.groupby('transaction_id')['purchase_sum'].sum().mean() # смотрим среднее и только по train_purch
mean_bill_sum_common = purches.groupby('transaction_id')['purchase_sum'].sum().mean() # и по train_purch + test_purch

CPU times: user 27.4 s, sys: 404 ms, total: 27.8 s
Wall time: 27.7 s


In [30]:
%%time
A_group_train = list() 
A_group_common = list() 
print('grouping clients ...')
grouped_by_clients = train_purch.drop('client_id', axis=1).groupby('client_id')[['transaction_id', 'purchase_sum']]
for i, (client_id, data) in enumerate(grouped_by_clients):
    cur_mean = (data.groupby('transaction_id')['purchase_sum'].sum() > mean_bill_sum_train).mean()
    if cur_mean > 0.5:
        A_group_train.append(client_id)
    if cur_mean > 0.5:
        A_group_common.append(client_id)
        
    if i % 5000 == 0:
        print(f'{100 * i / len(train):>.4}%', end='  ')  
print()

grouping clients ...
0.0%  3.571%  7.141%  10.71%  14.28%  17.85%  21.42%  25.0%  28.57%  32.14%  35.71%  39.28%  42.85%  46.42%  49.99%  53.56%  57.13%  60.7%  64.27%  67.84%  71.41%  74.99%  78.56%  82.13%  85.7%  89.27%  92.84%  96.41%  99.98%  
CPU times: user 2min 25s, sys: 1.42 s, total: 2min 26s
Wall time: 2min 27s


In [31]:
A_index_train = train.index.isin(A_group_train)
B_index_train = ~A_index_train

A_index_common = train.index.isin(A_group_common)
B_index_common = ~A_index_common

In [32]:
# обоих вариантов нет
uplift(train[B_index_train]) - uplift(train[A_index_train]), \
uplift(train[B_index_common]) - uplift(train[A_index_common])

(0.023081876007569058, 0.023081876007569058)

In [33]:
train_mean = train_purch['purchase_sum'].mean()
common_mean = purches['purchase_sum'].mean()
train_mean, common_mean # слишком похожи, так что можно взять любое

(775.8743808819485, 775.8723816644797)

In [34]:
%%time
A_group = list() 
B_group = list() 

grouped_by_clients = train_purch.drop('client_id', axis=1).groupby('client_id')['purchase_sum']
for i, (client_id, client_purches) in enumerate(grouped_by_clients):
    cur_mean = (client_purches > train_mean).mean()
    if  cur_mean > 0.5:
        A_group.append(client_id)
    elif cur_mean < 0.5:
        B_group.append(client_id)
    
    if i % 5000 == 0:
        print(f'{100 * i / len(train):>.4}%', end='  ')  
print()

0.0%  3.571%  7.141%  10.71%  14.28%  17.85%  21.42%  25.0%  28.57%  32.14%  35.71%  39.28%  42.85%  46.42%  49.99%  53.56%  57.13%  60.7%  64.27%  67.84%  71.41%  74.99%  78.56%  82.13%  85.7%  89.27%  92.84%  96.41%  99.98%  
CPU times: user 35.3 s, sys: 851 ms, total: 36.2 s
Wall time: 36.2 s


In [35]:
A_index = train.index.isin(A_group)
B_index = train.index.isin(B_group)

In [36]:
# снова нет такого варианта (идей тоже нет, так что выберу просто ближайший вариант)
uplift(train[B_index]) - uplift(train[A_index])

1.062045096093589

## P.S. 

#### Было бы классно, если бы можно было задать простые уточняющие вопросы типа "Что именно считать за дату настоящего момента?" или "Что понимается под чеком (сумму котрого надо считать): purchases или transaction?". 

#### А то много времени было потрачено на то, чтобы написать замороченные запросы и в итоге понять, что такого варината ответа в тесте близко нет, значит, имелось в виду что-то другое.